In [3]:
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain.vectorstores import Chroma
from langchain.chains import RetrievalQA
from pypdf import PdfReader
from langchain.document_loaders import PyPDFLoader
from langchain.llms import OpenAI
import faiss
from langchain_community.docstore.in_memory import InMemoryDocstore
from langchain_community.vectorstores import FAISS
from langchain_openai import ChatOpenAI

# SAH:
%pip install -U langchain-ollama
from langchain_ollama import OllamaEmbeddings
from langchain_community.embeddings import OllamaEmbeddings
from langchain_community.vectorstores import FAISS

Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
import json
import pandas as pd
import glob
from openai import OpenAI

In [5]:
# Get all PDFs inside the tsmc folder
pdf_files = glob.glob("./tsmc/*.pdf")

In [7]:
# Load & split documents from all PDFs
all_docs = []
text_splitter = RecursiveCharacterTextSplitter(chunk_size=500, chunk_overlap=50)

for pdf in pdf_files:
    loader = PyPDFLoader(pdf)
    data = loader.load()
    texts = text_splitter.split_documents(data)
    all_docs.extend(texts)

In [9]:
# SAH: API Solution requiring OpenAI API Key:
""" embeddings = OpenAIEmbeddings(model="text-embedding-3-large") """

# SAH - local solution:
embeddings = OllamaEmbeddings(model="mxbai-embed-large:335m")

In [ ]:
""" os.environ["OPENAI_API_KEY"] = "sk-XXX" #Replace with your OpenAI API key """

In [11]:
from langchain_core.vectorstores import InMemoryVectorStore

vector_store = InMemoryVectorStore.from_documents(all_docs, embeddings)

KeyboardInterrupt: 

In [10]:
print(f"✅ Total PDFs: {len(pdf_files)}")
print(f"✅ Total chunks stored: {len(all_docs)}")

✅ Total PDFs: 18
✅ Total chunks stored: 22476


In [11]:
first_chunk = all_docs[102]
print("=== Page Content ===")
print(first_chunk.page_content)
print("=== Metadata ===")
print(first_chunk.metadata)

=== Page Content ===
4.4 Compensation Committee
The Compensation Committee assists the Board in discharging 
its responsibilities related to TSMC’s compensation and benefits 
policies, plans and programs, and in the evaluation and compensation 
of TSMC’s executives. The Committee meets at least four times a 
year.
The Compensation Committee was comprised of four members. 
All three independent directors served as voting members of the 
Committee and the Chairman of the Board, Dr. Morris Chang,
=== Metadata ===
{'producer': 'Adobe Acrobat 8.0', 'creator': 'Adobe Acrobat 8.0 Combine Files', 'creationdate': '2011-12-07T17:43:42+08:00', 'moddate': '2011-12-07T17:48:52+08:00', 'source': './tsmc/TSMC CSR 2010.pdf', 'total_pages': 84, 'page': 15, 'page_label': '16'}


In [ ]:
#Use GPT-4o

llm = ChatOpenAI(
    model="gpt-4o",
    temperature=0,
    openai_api_key=os.environ.get("OPENAI_API_KEY")
)


In [ ]:
#Build RetrievalQA
qa_chain = RetrievalQA.from_chain_type(
    llm=llm,
    retriever=vector_store.as_retriever(),
    return_source_documents=True
)

In [ ]:
#Run a query
query = "Tell the key points about TSMC's 3nm technology from the documents."
result = qa_chain({"query": query})

print(result["result"])